In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scraping_class
from matplotlib.dates import DateFormatter
import seaborn as sns
from bs4 import BeautifulSoup

In [27]:
def get_metadata(movie_url):
    
    #connector = scraping_class.Connector(logfile)

    #response, call_id = connector.get(movie_url, 'get_metadata')
    response = requests.get(movie_url, headers={"Accept-Language":"en-US, en;q=0.5"})

    if response.ok:
    
        soup = BeautifulSoup(response.text,'html.parser')
        movie = soup.find_all('div',attrs={'class':'txt-block'})
        metadata = {}
        
        for i in movie:
            try:
                lookup = ['country', 'language', 'color']
                lookup2 = ['release date', 'aspect ratio']
                valuelist = ['budget', 'cumulative worldwide gross', 'aspect ratio']
                
                headline = i.find('h4').text.lower().split(':')[0]
                txtblock = i.text
                
                if headline in lookup:
                    link_values = i.find('a').text
                    metadata[headline] = link_values
                
                if headline in lookup2:
                    values = txtblock.split('\n')[1].split(':')[1]
                    try:
                        values = values.split('(')[0].strip()
                        metadata[headline] = values
                    except:
                        values = float(values.strip())
                        metadata[headline] = values
                        
                data = txtblock.split(':')[0].strip().lower()
                values = txtblock.split(':')[1].split('$')[1].strip().lower()
                
                if data in valuelist:
                    values = values.replace(',','')
                    
                    try:
                        values = int(values.split('(')[0])
                        metadata[data] = values
                        
                    except:
                        values = int(values)
                        metadata[data] = values
                        
            except:
                NameError

        return metadata
    
    else:
        return response.ok

In [69]:
link_list = ['https://www.imdb.com/title/tt0018578/', 'https://www.imdb.com/title/tt6966692/', "https://www.imdb.com/title/tt0018455/"]
df = [get_metadata(i) for i in link_list]

In [70]:
pd.DataFrame(df)

,aspect ratio,budget,color,country,cumulative worldwide gross,language,release date
0,1.33,2000000.0,Black and White,USA,746,English,5 January 1929
1,2.00,23000000.0,Color,USA,320940163,English,16 November 2018
2,1.33,NaN,Black and White,USA,121107,English,4 November 1927
